In [1]:
%matplotlib inline
import os
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as pltb

INPUT_FOLDER = '{}/workspace/data/BRATS2015_Training'.format(os.getenv("HOME"))

### Data structure

In [2]:
grades = ['HGG', 'LGG']
modalities = []
for grd in grades:
    pat_path = os.path.join(INPUT_FOLDER, grd)
    patients = os.listdir(pat_path)
    for pat in patients:
        mod_path = os.path.join(pat_path, pat)
        modalities = os.listdir(mod_path)
        # Each patient is recorded with modality T1, T1c, T2 and Flair
        # Groud truth segmentation is saved with 'more'
        

### Mean and std

In [3]:
import preprocessing.volume_image
from preprocessing.image_loader import BRATSDataLoader
from preprocessing.volume_image import VolumeImageDataGenerator
import yaml
with open("args.yml", 'r') as stream:
    try:
        config_args = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

Using TensorFlow backend.


In [4]:
def increment_mean(avg_prev, n_prev, batch):
    """Estimate mean of arrays with size
    bigger than memory limits ONLINE by small batches.
    # Arguments
        avg_prev: Incremental average computed by the values from the previous batches
        n_prev: Number of values used for computing the current mean_inc
        batch: Batch of values to update avg_inc, list-like
    # Returns
        avg_cur: Incremental average updated with the current batch
        n_cur: Number of values used for computing the current mean
    """
    batch = np.array(batch)
    avg_batch = np.mean(batch)
    n_batch = batch.size
    if not n_prev:
        return avg_batch, n_batch
    n_cur = n_prev + n_batch
    avg_inc = (n_batch / float(n_cur)) * (avg_batch - avg_prev)
    avg_cur = avg_prev + avg_inc
    return avg_cur, n_cur


def increment_var(var_prev, avg_prev, n_prev, batch):
    """Estimate mean and variance of arrays with size
    bigger than memory limits ONLINE using parallel methods by Chan et al. (1979)
    # Arguments
        var_prev: Incremental variance computed by the values from the previous batches
        avg_prev: Incremental average computed by the values from the previous batches
        n_prev: Number of values used previously
        batch: Batch of values to update the variance, an array or a list
    # Returns
        var_cur: Incremental variance updated with the current batch
        avg_cur: Incremental average updated with the current batch
        n_cur: Number of values used for computing the current variance
    """

    batch = np.array(batch)
    var_batch = np.var(batch)
    avg_batch = np.mean(batch)
    n_batch = batch.size
    if not n_prev:
        return var_batch, avg_batch, n_batch
    n_cur = n_prev + n_batch
    delta = avg_batch - avg_prev
    avg_inc = (n_batch / float(n_cur)) * delta
    avg_cur = avg_prev + avg_inc
    
    var_acc = var_prev * n_prev
    var_batch_acc = var_batch * n_batch
    var_inc = var_batch_acc + (delta ** 2) * n_prev * n_batch / n_cur
    var_acc += var_inc
    var_cur = var_acc / float(n_cur)
    return var_cur, avg_cur, n_cur

In [5]:
brats_loader = BRATSDataLoader(**config_args['brats_data_loader']['trainval'])
datagen = VolumeImageDataGenerator(**config_args['volume_image_data_generator']['train'])
iter_args = config_args['volume_image_data_generator']['flow_from_loader']
iter_args['volume_image_data_loader'] = brats_loader
iterator = datagen.flow_from_loader(**iter_args)

patients, classes = brats_loader.get_patients_classes()
len(patients)

In [6]:
# testing dataset as well
brats_testing_loader = BRATSDataLoader(**config_args['brats_data_loader']['test'])
test_patients, test_classes = brats_testing_loader.get_patients_classes()

In [ ]:
var = None
mean = None
num_voxels = None
for idx, p in enumerate(patients + test_patients):
    print("Reading {}. {}".format(idx, p))
    batch_x, batch_y = iterator.next()
    var, mean, num_voxels = increment_var(var, mean, num_voxels, batch_x)
print("Std", np.sqrt(var))
print("Mean", mean)
print("Num voxels", num_voxels)

Reading 0. ../data/BRATS2015_Training/LGG/brats_tcia_pat393_0001/VSD.Brain.XX.O.MR_T1c.41090/VSD.Brain.XX.O.MR_T1c.41090.mha
Reading 1. ../data/BRATS2015_Training/LGG/brats_tcia_pat393_0001/VSD.Brain.XX.O.MR_Flair.41088/VSD.Brain.XX.O.MR_Flair.41088.mha
Reading 2. ../data/BRATS2015_Training/LGG/brats_tcia_pat393_0001/VSD.Brain.XX.O.MR_T2.41089/VSD.Brain.XX.O.MR_T2.41089.mha
Reading 3. ../data/BRATS2015_Training/LGG/brats_tcia_pat393_0001/VSD.Brain.XX.O.MR_T1.41091/VSD.Brain.XX.O.MR_T1.41091.mha
Reading 4. ../data/BRATS2015_Training/LGG/brats_tcia_pat130_0001/VSD.Brain.XX.O.MR_T2.40832/VSD.Brain.XX.O.MR_T2.40832.mha
Reading 5. ../data/BRATS2015_Training/LGG/brats_tcia_pat130_0001/VSD.Brain.XX.O.MR_Flair.40831/VSD.Brain.XX.O.MR_Flair.40831.mha
Reading 6. ../data/BRATS2015_Training/LGG/brats_tcia_pat130_0001/VSD.Brain.XX.O.MR_T1.40834/VSD.Brain.XX.O.MR_T1.40834.mha
Reading 7. ../data/BRATS2015_Training/LGG/brats_tcia_pat130_0001/VSD.Brain.XX.O.MR_T1c.40833/VSD.Brain.XX.O.MR_T1c.40833.mha
